# Assignment 3
## INFS 770
## Jeremiah Lowhorn
## 3/25/2019

In [112]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer 
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import gensim
from gensim.models import LdaModel, LsiModel
from gensim.models import CoherenceModel

# Part 1.
Each sentence in the review.txt file is a review. Please write code to create a list of strings (call this list “docs”), and each component of the list is a review. Please print the list. 

In [86]:
text_file = open("reviews.txt", "r")
docs = text_file.read().split('\n')

In [87]:
print(docs)

['with all the available applications and new updates for its software , iphone is a phone you grow with , not out of .', 'i upgraded from the sd550 canon there is not comparisent  .', 'i have also converted my office from the pc and windows world to mac os and that is the best business decision i have made in years i have tried other phones with android , windows mobile and they all fall short  .', "i've also experienced the unit occasionally freezing or resetting itself , usually when the bluetooth connection is active and i'm receiving a phone call  .", "this is my family's seventh (at least) garmin unit , and may be our last .", 'thanks canon for making my family happy and able to share our family photos again  .', 'a really great camera  .', 'so i knew when i is buying this unit i is not buying a unit that i would build for this price but the best available as far as it being balanced in   , efficientcy and bells and whisles  .', '(i have one , and i will also be getting the iphon

In [88]:
print(docs[0])

with all the available applications and new updates for its software , iphone is a phone you grow with , not out of .


# Part 2
Write code to tokenize the docs using the TF-IDF vectorizer (You need to copy and modify the corresponding code in the preprocessing.ipynb and topic modeling.ipynb file I used in lecture on 3/18). In this task you need to 1) remove punctuations, 2) remove numbers, 3) do a lemmatization, 4) remove stop words, 5) remove words that appear in only one document, and 6) remove words that appears in over 90% of documents (When you do TFIDF, you often need to ignore words that too common or too rare). You don’t need to lower case the texts as they have already been lower-cased. In this task, you use the default value of norm (i.e., norm = None) when you do TF-IDF.  Please print the features (i.e., vectorizer.vocabulary_) extracted based on TF-IDF. 

In [89]:
def before_token(documents):
    # conver words to lower case
    lower = map(str.lower, documents)
    # remove puntuations
    punctuationless = list(map(lambda x: " ".join(re.findall('\\b\\w\\w+\\b',x)),lower ))
    # remove numbers
    return list(map(lambda x:re.sub('\\b[0-9]+\\b', '', x), punctuationless))
docs1 = before_token(docs)
print(docs1)

['with all the available applications and new updates for its software iphone is phone you grow with not out of', 'upgraded from the sd550 canon there is not comparisent', 'have also converted my office from the pc and windows world to mac os and that is the best business decision have made in years have tried other phones with android windows mobile and they all fall short', 've also experienced the unit occasionally freezing or resetting itself usually when the bluetooth connection is active and receiving phone call', 'this is my family seventh at least garmin unit and may be our last', 'thanks canon for making my family happy and able to share our family photos again', 'really great camera', 'so knew when is buying this unit is not buying unit that would build for this price but the best available as far as it being balanced in efficientcy and bells and whisles', 'have one and will also be getting the iphone 4g as soon as it comes out as well it is by far the best mobile that is cur

In [90]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t,"v") for t in word_tokenize(doc)]
stopwords = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jlowh001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [127]:
#two additional arguments have to be set for the maximum word frequency in the documents and for removing rare words
# max_df and min_df respectively
vectorizer1 = TfidfVectorizer(tokenizer=LemmaTokenizer(),norm=None,stop_words=stopwords,max_df=0.9,min_df=2)
#vectorizer = TfidfVectorizer(tokenizer=LemmaTokenizer(),stop_words=stopwords)
corpus_vect1 = vectorizer1.fit_transform(docs1)
print(corpus_vect1) # sparse matrix

  (0, 28)	5.3694478524670215
  (0, 23)	5.3694478524670215
  (0, 188)	4.388618599455295
  (0, 309)	4.858622228701031
  (0, 273)	5.081765780015241
  (0, 150)	2.971552579668651
  (0, 211)	3.034072936649985
  (0, 134)	5.3694478524670215
  (1, 310)	5.3694478524670215
  (1, 252)	5.3694478524670215
  (1, 53)	4.165475048141086
  (2, 211)	3.034072936649985
  (2, 13)	4.070164868336761
  (2, 206)	5.3694478524670215
  (2, 326)	4.676300671907076
  (2, 37)	4.070164868336761
  (2, 47)	5.081765780015241
  (2, 170)	4.165475048141086
  (2, 330)	5.081765780015241
  (2, 305)	5.3694478524670215
  (2, 16)	5.3694478524670215
  (2, 177)	5.3694478524670215
  (3, 211)	3.034072936649985
  (3, 13)	4.070164868336761
  (3, 106)	4.388618599455295
  :	:
  (230, 284)	4.858622228701031
  (230, 171)	5.081765780015241
  (231, 211)	3.034072936649985
  (231, 170)	4.165475048141086
  (231, 327)	3.760009940032921
  (231, 199)	3.523621161968691
  (231, 257)	4.676300671907076
  (231, 312)	3.034072936649985
  (231, 166)	3.90311

In [128]:
df_vect1 = pd.DataFrame(corpus_vect1.toarray(), columns=vectorizer1.get_feature_names())
print(df_vect1)

           3g      able    access  account  accurate  actually   ad  addition  \
0    0.000000  0.000000  0.000000      0.0       0.0  0.000000  0.0       0.0   
1    0.000000  0.000000  0.000000      0.0       0.0  0.000000  0.0       0.0   
2    0.000000  0.000000  0.000000      0.0       0.0  0.000000  0.0       0.0   
3    0.000000  0.000000  0.000000      0.0       0.0  0.000000  0.0       0.0   
4    0.000000  0.000000  0.000000      0.0       0.0  0.000000  0.0       0.0   
5    0.000000  4.858622  0.000000      0.0       0.0  0.000000  0.0       0.0   
6    0.000000  0.000000  0.000000      0.0       0.0  0.000000  0.0       0.0   
7    0.000000  0.000000  0.000000      0.0       0.0  0.000000  0.0       0.0   
8    0.000000  0.000000  0.000000      0.0       0.0  0.000000  0.0       0.0   
9    0.000000  0.000000  0.000000      0.0       0.0  0.000000  0.0       0.0   
10   0.000000  0.000000  0.000000      0.0       0.0  0.000000  0.0       0.0   
11   0.000000  0.000000  0.0

# Part 3
Create a textbox and use your own language to discuss what TF-IDF means. 

TF-IDF is the inverse document frequency. It essentially assigns a weight to words that appear most frequently in the corpus being analyzed.  The TF-IDF value increases as the number of times a word appears in a document and is offset by the number of documents that contain the word in the corpus. 

# Part 4
Build a topic model using Latent Dirichlet Allocation (LDA) and the vectorised data obtained from T2. Set the number of topics to 3. Please write code to print the topics. At this stage, you don’t need to write code to show the topic/document table. 

In [129]:
# convert the vectorized data to a gensim corpus object
corpus1 = gensim.matutils.Sparse2Corpus(corpus_vect1, documents_columns=False)
id2word1 = dict((v,k) for k,v in vectorizer1.vocabulary_.items())
print(id2word1)

{28: 'available', 23: 'applications', 188: 'new', 309: 'update', 273: 'software', 150: 'iphone', 211: 'phone', 134: 'grow', 310: 'upgrade', 252: 'sd550', 53: 'canon', 13: 'also', 206: 'pc', 326: 'world', 37: 'best', 47: 'business', 170: 'make', 330: 'years', 305: 'try', 16: 'android', 177: 'mobile', 106: 'experience', 307: 'unit', 121: 'freeze', 241: 'reset', 315: 'usually', 43: 'bluetooth', 237: 'receive', 50: 'call', 108: 'family', 157: 'least', 124: 'garmin', 155: 'last', 291: 'thank', 137: 'happy', 1: 'able', 262: 'share', 214: 'photos', 235: 'really', 133: 'great', 51: 'camera', 153: 'know', 48: 'buy', 327: 'would', 46: 'build', 223: 'price', 110: 'far', 34: 'balance', 199: 'one', 127: 'get', 321: 'well', 83: 'currently', 196: 'offer', 257: 'see', 58: 'carry', 180: 'month', 84: 'data', 218: 'plan', 319: 'way', 41: 'blackberry', 90: 'digital', 296: 'time', 255: 'sd790is', 98: 'easy', 27: 'auto', 178: 'mode', 18: 'anyone', 318: 'want', 314: 'users', 184: 'multitasking', 40: 'big', 8

In [130]:
# build the lda model
lda1 = LdaModel(corpus1, num_topics=3,id2word=id2word1, passes=10)
print(lda1.print_topics())

[(0, '0.020*"iphone" + 0.020*"get" + 0.020*"well" + 0.019*"phone" + 0.015*"one" + 0.015*"use" + 0.014*"garmin" + 0.014*"traffic" + 0.014*"happy" + 0.013*"also"'), (1, '0.023*"camera" + 0.022*"picture" + 0.019*"screen" + 0.019*"quality" + 0.016*"take" + 0.015*"love" + 0.014*"think" + 0.014*"great" + 0.013*"review" + 0.013*"get"'), (2, '0.021*"use" + 0.019*"iphone" + 0.018*"like" + 0.018*"best" + 0.017*"phone" + 0.015*"say" + 0.015*"recommend" + 0.015*"would" + 0.014*"easy" + 0.014*"camera"')]


In [131]:
#calculate the coeherence score https://radimrehurek.com/gensim/models/coherencemodel.html
cm = CoherenceModel(model=lda1, corpus=corpus1, coherence='u_mass')
coherence = cm.get_coherence()
coherence

-12.327479884872167

# Part 5  
Build a topic model using Latent Dirichlet Allocation (LDA) and the vectorised data from T2. Set the number of topics to 4. Please write code to print the topics. You don’t need to write code to show the topic/document table. 

In [132]:
# build the lda model
lda2 = LdaModel(corpus1, num_topics=4,id2word=id2word1, passes=10)
print(lda2.print_topics())


[(0, '0.031*"great" + 0.030*"picture" + 0.025*"camera" + 0.019*"take" + 0.018*"recommend" + 0.015*"price" + 0.015*"anyone" + 0.013*"screen" + 0.013*"right" + 0.013*"use"'), (1, '0.029*"phone" + 0.022*"iphone" + 0.020*"love" + 0.019*"say" + 0.019*"buy" + 0.019*"battery" + 0.018*"would" + 0.018*"use" + 0.015*"also" + 0.014*"still"'), (2, '0.020*"use" + 0.020*"iphone" + 0.020*"well" + 0.018*"like" + 0.018*"phone" + 0.017*"way" + 0.017*"camera" + 0.016*"get" + 0.016*"traffic" + 0.015*"one"'), (3, '0.018*"rout" + 0.018*"time" + 0.016*"iphone" + 0.016*"much" + 0.016*"one" + 0.015*"good" + 0.014*"sd790is" + 0.013*"get" + 0.013*"canon" + 0.013*"offer"')]


In [133]:
#calculate the coeherence score https://radimrehurek.com/gensim/models/coherencemodel.html
cm = CoherenceModel(model=lda2, corpus=corpus1, coherence='u_mass')
coherence = cm.get_coherence()
coherence

-13.797193316560879

# Part 6
Build a topic model using Latent Dirichlet Allocation (LDA) and the vectorised data from T2. Set the number of topics to 5. Please write code to print the topics. You don’t need to write code to show the topic/document table. 

In [134]:
# build the lda model
lda3 = LdaModel(corpus1, num_topics=5,id2word=id2word1, passes=10)
print(lda3.print_topics())

[(0, '0.026*"screen" + 0.023*"get" + 0.022*"camera" + 0.022*"much" + 0.021*"go" + 0.017*"route" + 0.017*"price" + 0.015*"video" + 0.015*"iphone" + 0.015*"amaze"'), (1, '0.041*"phone" + 0.021*"like" + 0.021*"would" + 0.020*"iphone" + 0.020*"apple" + 0.019*"take" + 0.018*"experience" + 0.018*"know" + 0.018*"use" + 0.017*"way"'), (2, '0.028*"traffic" + 0.021*"recommend" + 0.020*"picture" + 0.017*"would" + 0.017*"shoot" + 0.016*"battery" + 0.016*"point" + 0.016*"use" + 0.015*"life" + 0.015*"camera"'), (3, '0.031*"phone" + 0.029*"happy" + 0.024*"iphone" + 0.024*"use" + 0.021*"multi" + 0.018*"task" + 0.017*"family" + 0.017*"able" + 0.017*"google" + 0.016*"apps"'), (4, '0.034*"great" + 0.032*"camera" + 0.031*"well" + 0.026*"buy" + 0.019*"could" + 0.018*"best" + 0.014*"like" + 0.013*"always" + 0.012*"look" + 0.012*"nuvi"')]


In [135]:
#calculate the coeherence score https://radimrehurek.com/gensim/models/coherencemodel.html
cm = CoherenceModel(model=lda3, corpus=corpus1, coherence='u_mass')
coherence = cm.get_coherence()
coherence

-14.5377955157739

# Part 7
Create a textbox and tell me among T4, T5, T6 results, which number of topics (among 3,4,5) gives you the best topic modeling results. 

Rather than trying to assume which model performed the best I used something called a coherence score from the gensim library. According to the definition of coherence, the model with the highest score should indicate the best model. In our case the model with 3 topics has the highest coherence score of -12.32 compared to 4 topcs (-13.79) and 5 topics (-14.54).

# Part 8
Please re-run the TF-IDF vectorizer. This time set norm to be “l1” for the vectorizer. Then set num_topics to be the optimal number of topics you found in T7 and re-run LDA. Please write code to print the topics. You don’t need to write code to show the topic/document table. 

In [136]:
#two additional arguments have to be set for the maximum word frequency in the documents and for removing rare words
# max_df and min_df respectively
vectorizer2 = TfidfVectorizer(tokenizer=LemmaTokenizer(),norm='l1',stop_words=stopwords,max_df=0.9,min_df=2)
#vectorizer = TfidfVectorizer(tokenizer=LemmaTokenizer(),stop_words=stopwords)
corpus_vect2 = vectorizer2.fit_transform(docs1)

corpus2 = gensim.matutils.Sparse2Corpus(corpus_vect2, documents_columns=False)
id2word2 = dict((v,k) for k,v in vectorizer2.vocabulary_.items())

In [137]:
# build the lda model
lda4 = LdaModel(corpus2, num_topics=3,id2word=id2word2, passes=10)
print(lda4.print_topics())

[(0, '0.028*"iphone" + 0.018*"get" + 0.016*"use" + 0.012*"recommend" + 0.010*"much" + 0.010*"happy" + 0.010*"know" + 0.010*"point" + 0.010*"camera" + 0.010*"easy"'), (1, '0.017*"picture" + 0.014*"phone" + 0.012*"take" + 0.012*"one" + 0.011*"traffic" + 0.011*"best" + 0.010*"garmins" + 0.010*"screen" + 0.009*"route" + 0.008*"flash"'), (2, '0.029*"great" + 0.014*"camera" + 0.010*"canon" + 0.009*"weeks" + 0.009*"far" + 0.009*"also" + 0.008*"happy" + 0.008*"unit" + 0.008*"garmin" + 0.007*"work"')]


In [138]:
#calculate the coeherence score https://radimrehurek.com/gensim/models/coherencemodel.html
cm = CoherenceModel(model=lda4, corpus=corpus2, coherence='u_mass')
coherence = cm.get_coherence()
coherence

-15.960593880385744

# Part 9
Please re-run the TF-IDF vectorizer. This time set norm to be “l2” for the vectorizer. Then set num_topics to be the optimal number of topics you found in T7 and re-run LDA. Please write code to print the topics. You don’t need to write code to show the topic/document table. 

In [139]:
#two additional arguments have to be set for the maximum word frequency in the documents and for removing rare words
# max_df and min_df respectively
vectorizer3 = TfidfVectorizer(tokenizer=LemmaTokenizer(),norm='l2',stop_words=stopwords,max_df=0.9,min_df=2)
#vectorizer = TfidfVectorizer(tokenizer=LemmaTokenizer(),stop_words=stopwords)
corpus_vect3 = vectorizer3.fit_transform(docs1)

corpus3 = gensim.matutils.Sparse2Corpus(corpus_vect3, documents_columns=False)
id2word3 = dict((v,k) for k,v in vectorizer3.vocabulary_.items())

In [140]:
# build the lda model
lda5 = LdaModel(corpus3, num_topics=3,id2word=id2word3, passes=10)
print(lda5.print_topics())

[(0, '0.020*"iphone" + 0.018*"phone" + 0.017*"great" + 0.012*"know" + 0.012*"point" + 0.011*"shoot" + 0.010*"use" + 0.010*"also" + 0.009*"route" + 0.009*"traffic"'), (1, '0.020*"get" + 0.017*"use" + 0.015*"happy" + 0.014*"easy" + 0.014*"go" + 0.013*"iphone" + 0.012*"picture" + 0.012*"phone" + 0.011*"take" + 0.010*"much"'), (2, '0.025*"camera" + 0.017*"recommend" + 0.015*"best" + 0.012*"great" + 0.012*"say" + 0.012*"big" + 0.011*"anyone" + 0.010*"would" + 0.009*"service" + 0.008*"iphone"')]


In [142]:
#calculate the coeherence score https://radimrehurek.com/gensim/models/coherencemodel.html
cm = CoherenceModel(model=lda5, corpus=corpus3, coherence='u_mass')
coherence = cm.get_coherence()
coherence

-13.930024825029756

# Part 10 
Please create a textbox and compare the topics you obtained with norm set to be 1) None, 2) “l1” or 3) “l2” (num_topics is always the optimal number of topics you identified in T7). Please tell me which norm gives you the best topics, and why.

Based upon the coherence score, the model with no norm seems to provide the best topics. The first topic in the first model seems to be a positive topic regarding the iphone, the second topic seems to indicate the camera and the picture of the phone is of great quality, and the third is not perfectly clear. In the second model, l1 norm, the first topic indicates that the customers are pleased with the iphone, the second indicates the quality of the picture of the phone, and the third indicates positive sentiment on the camera. The third model the first topic indicates the sentiment of the iphone, the second topic seems to indicate the ease of use of the iphone, and the third indicates the positive sentiment of the camera. By examining the topics manually I get a sense that the topic model with l2 norm actually produced more coherent topics than the model with no norm even though the coherence score was slightly lower. 

# Part 11
In part 11, you have identified the best norm and in T7, you have identified the best number of topics. Now please continue to write code to show the topic/document table of LDA with the best number of topics using the vectorized dataset obtained with the best norm.

In [143]:
# use the lda model to transform documents
lda_docs = lda5[corpus3]
for row in lda_docs:
    print(row)

[(0, 0.13840547), (1, 0.7665947), (2, 0.09499984)]
[(0, 0.7270951), (1, 0.12863052), (2, 0.1442743)]
[(0, 0.13489498), (1, 0.08892907), (2, 0.7761759)]
[(0, 0.82943296), (1, 0.087035894), (2, 0.08353114)]
[(0, 0.11292873), (1, 0.77843934), (2, 0.10863196)]
[(0, 0.09499653), (1, 0.7121409), (2, 0.19286254)]
[(0, 0.13798265), (1, 0.2075175), (2, 0.6544998)]
[(0, 0.16589737), (1, 0.4507372), (2, 0.38336536)]
[(0, 0.12969916), (1, 0.50377625), (2, 0.36652458)]
[(0, 0.4982921), (1, 0.40527695), (2, 0.09643095)]
[(0, 0.11486019), (1, 0.11845207), (2, 0.76668775)]
[(0, 0.11342571), (1, 0.7714364), (2, 0.115137935)]
[(0, 0.11918351), (1, 0.09179381), (2, 0.7890226)]
[(0, 0.09236302), (1, 0.81589055), (2, 0.09174641)]
[(0, 0.16733862), (1, 0.16729075), (2, 0.66537064)]
[(0, 0.19285905), (1, 0.6800263), (2, 0.12711468)]
[(0, 0.10901905), (1, 0.7781356), (2, 0.11284535)]
[(0, 0.7349966), (1, 0.1338956), (2, 0.13110778)]
[(0, 0.6042523), (1, 0.25922796), (2, 0.13651977)]
[(0, 0.2788655), (1, 0.091

In [144]:
# extract the scores and round them to 3 decimal places
scores = np.round([[doc[1] for doc in row] for row in lda_docs], 3)
print(scores)

[[0.139 0.766 0.095]
 [0.727 0.129 0.144]
 [0.135 0.089 0.776]
 [0.829 0.087 0.084]
 [0.113 0.778 0.109]
 [0.095 0.712 0.193]
 [0.138 0.208 0.654]
 [0.166 0.451 0.383]
 [0.13  0.504 0.367]
 [0.498 0.405 0.096]
 [0.115 0.118 0.767]
 [0.113 0.771 0.115]
 [0.119 0.092 0.789]
 [0.092 0.816 0.092]
 [0.167 0.167 0.665]
 [0.226 0.647 0.127]
 [0.109 0.778 0.113]
 [0.735 0.134 0.131]
 [0.605 0.259 0.137]
 [0.279 0.092 0.63 ]
 [0.139 0.139 0.722]
 [0.124 0.75  0.126]
 [0.148 0.701 0.151]
 [0.644 0.176 0.18 ]
 [0.079 0.853 0.067]
 [0.78  0.107 0.113]
 [0.747 0.137 0.116]
 [0.783 0.106 0.111]
 [0.332 0.541 0.127]
 [0.143 0.738 0.119]
 [0.087 0.085 0.829]
 [0.105 0.102 0.793]
 [0.187 0.157 0.656]
 [0.78  0.11  0.111]
 [0.73  0.14  0.13 ]
 [0.768 0.119 0.113]
 [0.706 0.197 0.098]
 [0.097 0.808 0.095]
 [0.716 0.122 0.162]
 [0.151 0.144 0.705]
 [0.333 0.333 0.333]
 [0.778 0.137 0.085]
 [0.136 0.73  0.134]
 [0.663 0.143 0.193]
 [0.107 0.772 0.121]
 [0.162 0.691 0.147]
 [0.741 0.13  0.129]
 [0.091 0.813

In [145]:
# convert the documents scores into a data frame
df_lda = pd.DataFrame(scores, columns=["topic 1", "topic 2",'topic 3'])
df_lda

,topic 1,topic 2,topic 3
0,0.139,0.766,0.095
1,0.727,0.129,0.144
2,0.135,0.089,0.776
3,0.829,0.087,0.084
4,0.113,0.778,0.109
5,0.095,0.712,0.193
6,0.138,0.208,0.654
7,0.166,0.451,0.383
8,0.130,0.504,0.367
9,0.498,0.405,0.096


# Part 12
Run truncated SVD(set n_components to be the optimal number of topics you found in T7) and print topics and the topic/document tab

In [148]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=3)
tsvd.fit(corpus_vect3)
print(np.round(tsvd.transform(corpus_vect3), 3))
print(tsvd.singular_values_)

[[ 0.218 -0.237  0.124]
 [ 0.028  0.024 -0.025]
 [ 0.168 -0.111 -0.004]
 [ 0.157 -0.122 -0.019]
 [ 0.068 -0.001 -0.132]
 [ 0.069  0.018 -0.063]
 [ 0.396  0.52   0.289]
 [ 0.189 -0.016 -0.174]
 [ 0.285 -0.144 -0.081]
 [ 0.203 -0.182  0.014]
 [ 0.161  0.172  0.034]
 [ 0.088  0.06  -0.101]
 [ 0.21  -0.115 -0.111]
 [ 0.447  0.348 -0.211]
 [ 0.013  0.024 -0.002]
 [ 0.232 -0.22   0.066]
 [ 0.088  0.029 -0.122]
 [ 0.625 -0.143  0.531]
 [ 0.273  0.311  0.005]
 [ 0.33  -0.308  0.085]
 [ 0.066 -0.032  0.041]
 [ 0.061 -0.006 -0.054]
 [ 0.075 -0.015 -0.098]
 [ 0.021  0.012 -0.009]
 [ 0.236  0.15  -0.202]
 [ 0.279 -0.306  0.187]
 [ 0.188 -0.201  0.193]
 [ 0.198  0.207  0.092]
 [ 0.153  0.146 -0.199]
 [ 0.309  0.332 -0.055]
 [ 0.078 -0.003 -0.086]
 [ 0.078 -0.017 -0.018]
 [ 0.281 -0.3    0.274]
 [ 0.138 -0.115  0.033]
 [ 0.26   0.299  0.06 ]
 [ 0.03  -0.006 -0.035]
 [ 0.178  0.096 -0.143]
 [ 0.061  0.004 -0.136]
 [ 0.439 -0.395  0.078]
 [ 0.274 -0.225 -0.005]
 [ 0.     0.     0.   ]
 [ 0.287 -0.308 

In [149]:
df_comp = pd.DataFrame(tsvd.components_, columns=vectorizer.get_feature_names())
df_comp = df_comp.apply(lambda x: np.round(x,3))
print(df_comp)

      3g   able  access  account  accurate  actually     ad  addition    ads  \
0  0.012  0.022   0.019    0.013     0.013     0.015  0.008     0.032  0.010   
1  0.005  0.009  -0.013    0.003     0.014    -0.003 -0.001     0.013 -0.002   
2  0.008  0.016  -0.011   -0.043     0.004    -0.021 -0.030     0.013 -0.023   

     ago  ...     wish  without   work  world  would   yeah   year  years  \
0  0.016  ...    0.006    0.058  0.031  0.054  0.116  0.011  0.027  0.018   
1  0.016  ...   -0.000   -0.063  0.024 -0.062 -0.061  0.006 -0.028 -0.009   
2 -0.009  ...   -0.019    0.058  0.027  0.032 -0.017 -0.028  0.001 -0.017   

     yes    yet  
0  0.018  0.028  
1 -0.017  0.011  
2 -0.014 -0.009  

[3 rows x 333 columns]


## Top 10 Words from First Topic

In [164]:
cols = df_comp.columns.values.tolist()
vals_1 = df_comp.loc[0,:].values.tolist()
val_dict1 = dict(zip(cols, vals_1))

In [171]:
#top 10 words from first topic
sorted(val_dict1, key=val_dict1.get, reverse=True)[:10]

['great',
 'iphone',
 'phone',
 'camera',
 'use',
 'get',
 'picture',
 'go',
 'one',
 'take']

## Top 10 Words From Second Topic

In [172]:
vals_2 = df_comp.loc[1,:].values.tolist()
val_dict2 = dict(zip(cols, vals_2))

In [173]:
#top 10 words from second topic
sorted(val_dict2, key=val_dict2.get, reverse=True)[:10]

['great',
 'camera',
 'picture',
 'shoot',
 'point',
 'take',
 'really',
 'sd790is',
 'buy',
 'go']

## Top 10 Words From Third Topic

In [174]:
vals_3 = df_comp.loc[2,:].values.tolist()
val_dict3 = dict(zip(cols, vals_3))

In [175]:
#top 10 words from second topic
sorted(val_dict3, key=val_dict3.get, reverse=True)[:10]

['great',
 'iphone',
 'camera',
 'phone',
 'shoot',
 'apps',
 'design',
 'recommend',
 'point',
 'without']